In [2]:
import mosek
import numpy as np
from mosek import *
import mosek.fusion
from mosek.fusion import *
from math import *
from numpy import *
from numpy.random import *
import sys


**Newsvendor robusto

$\Omega$: Conjunto de escenarios (Subconjunto del conjunto de posibles demandas)
$\mathcal{P}$: Conjunto de ambiguedad de las distribuciones sobre $\Omega$

$$\underset{x\in \mathbb{R}_+}{min}\underset{\textbf{q}\in \mathcal{P}}{\text{max}} \;\; cx-p\sum_{\omega\in A}q_{w}\text{min}\{\omega,x\}\;$$



In [6]:
#Parámetros

#Vector de escenarios
muestra=poisson(50,150)
Di=muestra.tolist()
Di.sort()

D=[Di[0]]
Pt=ones(size(Di))/size(Di)
P=[Pt[0]]
j=0

for i in range(1,size(Di)):
    if Di[i]!=D[j]:
        D.append(Di[i])
        P.append(Pt[i])
        j=j+1
    else:
        P[j]=P[j]+Pt[i]

        
s=size(D)


#c: Costo del producto & p: precio
c=5
p=12


eps=0.1
inf=10000


In [7]:
def gamma(A):
    probA=0 
    for i in range(size(A)):
        cual=D.index(A[i])
        probA=probA+P[cual] 
        
    gam=2*(1-probA+eps*probA)
    return gam

In [8]:
#Función que retorna el valor óptimo del problema newsvendor no robusto 

def objetivo(x,D,P):
    suma=0
    for i in range(size(D)):
        suma=suma+(min(D[i],x))*P[i]
        
    A=c*x-p*suma
    return A




For a given $x\in \mathbb{R}_+$ and a subset $A$ of $\Omega$ we define $$f(x,A)=\underset{\textbf{q}\in \mathcal{P}(A)}{\text{max}} \;\; cx-p\sum_{\omega\in A}q_{w}\text{min}\{\omega,x\}\;,$$ where the ambiguity set $\mathcal{P}(A)$ consists of the probability measures on $\Omega$ supported on $A$ such that $||\textbf{q}-\theta||_{VT}\leq \epsilon$. 

Given $x\in \mathbb{R}_+$ and $A\subset \Omega$, $f(x,A)$ can be found y solving the following linear optimization problem: 


\begin{equation}
\begin{aligned}
   \underset{s,t,\textbf{q}\in \mathbb{R}^{|\Omega|}}{\text{max}} & \quad -p\sum_{\omega\in A}q_{\omega}\text{min}\{\omega,x\} \\
    \text{s.t.} \;\; & \mathbbm{1}_A^T\textbf{q}=1\\
    \; & \mathbbm{1}_{A^c}^T \textbf{q}=0\\
    \; & \textbf{q}-\theta \leq s\\
    \; & \theta-\textbf{q}\leq t\\
    \; & \mathbbm{1}^T(s+t)\leq 2\epsilon\\
    \; & s,t,\textbf{q} \geq \textbf{0}\\
\end{aligned}
\label{f(x,A)}
\end{equation}\\

In order to solve the problem $$\underset{x\in \mathbb{R}_+}{\text{min}}\left\{\underset{\textbf{q}\in \mathcal{P}(A)}{\text{max}}\; cx-p\sum_{\omega\in A}q_{\omega}\text{min}\{\omega,x\} \right\}$$ we need to use the dual form of problem (\ref{f(x,A)}). This can be done due to Theorem \ref{dualidad fuerte}.

\begin{equation}
\begin{aligned}
  \underset{\delta_1,\delta_2 \in \mathbb{R}^{|\Omega|}}{\underset{\gamma_1,\gamma_2, \lambda \in \mathbb{R}}{\text{min}}}  & \quad \gamma_1 + \delta_1^T\theta-\delta_2^T\theta+2\epsilon\lambda\\
    \text{s.t.} \;\; & \textbf{q}:\gamma_1\mathbbm{1}_A+\gamma_2\mathbbm{1}_{A^c}+\delta_1-\delta_2\geq -pz\\
    \; & s:\lambda\mathbbm{1}-\delta_1\geq \textbf{0}\\
    \; & t:\lambda\mathbbm{1}-\delta_2\geq \textbf{0}\\
    \; & \lambda,\delta_1(i),\delta_2(i) \geq 0, \quad i=1,\dots,|\Omega|\\
\end{aligned}
\label{dual de f(x,A)}
\end{equation}\\


In this problem formulation $z$ corresponds to the vector of size $|\Omega|$ whose $i$-th entry is defined by $z(i):=\text{min}\{x,\omega_i\}\cdot\mathbbm{1}_A(\omega_i)$.


In [9]:
#Definición de la función f(x,A)

def funcionf(x,A):

    s=size(D)
    
    probA=0 
    for i in range(size(A)):
        cual=D.index(A[i])
        probA=probA+P[cual]
        
    indicA=zeros(s)
    
    Q=zeros(s)
    for i in range(size(A)):
        cual=D.index(A[i])
        indicA[cual]=1
        Q[cual]=P[cual]/probA
    O=Q.tolist()
    
    O=P
    epsilon=gamma(A)
    
    matrizA=ones((3*s,3+2*s))

    with mosek.Env() as env:
            with env.Task(0, 0) as task:

                # Bound keys for constraints
                bkc = [mosek.boundkey.lo]*3*s   

                # Bound values for constraints
                menosPYs=[]
                for i in range(s):
                    menosPYs.append(-p*min(x,D[i])*indicA[i])

                blc = menosPYs+[0]*s*2
                buc = [inf]*3*s

                # Bound keys for variables
                bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1) 

                # Bound values for variables
                blx = [-inf]*2+[0]*(2*s+1)
                bux = [+inf]*(3+2*s)

                # Objective coefficients
                #coef5=round(epsilon*s/size(A),0)
                mO=[]
                for i in range(size(O)):
                    mO.append(-O[i])

                coef = [1]+[0]+[epsilon]+O+mO
                
                task.putcfix(c*x)

                # Below is the sparse representation of the A
                # matrix stored by column.

                numvar = len(bkx)
                numcon = len(bkc)

                # Append 'numcon' empty constraints.
                # The constraints will initially have no bounds.
                task.appendcons(numcon)

                # Append 'numvar' variables.
                # The variables will initially be fixed at zero (x=0).
                task.appendvars(numvar)

                #Append constraint matrix 
                task.putaijlist(range(0,s),[0]*s,indicA)
                task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
                task.putaijlist(range(0,s),range(3,3+s),ones(s))
                task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s))
                task.putaijlist(range(s,2*s),[2]*s,ones(s))
                task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
                task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
                task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))

                for i in range(numcon):
                    for j in range(numvar):
                        matrizA[i,j]=task.getaij(i,j) 


                for j in range(numvar):
                    # Set the linear term c_j in the objective.
                    task.putcj(j, coef[j])

                    # Set the bounds on variable j
                    # blx[j] <= x_j <= bux[j]
                    task.putvarbound(j, bkx[j], blx[j], bux[j])  


                # Set the bounds on constraints.
                # blc[i] <= constraint_i <= buc[i]
                for i in range(numcon):
                    task.putconbound(i, bkc[i], blc[i], buc[i])

                # Input the objective sense (minimize/maximize)
                task.putobjsense(mosek.objsense.minimize)

                # Solve the problem
                task.optimize()

                # Print a summary containing information
                # about the solution for debugging purposes
                resumen=task.solutionsummary(mosek.streamtype.msg)

                # Get status information about the solution
                solsta = task.getsolsta(mosek.soltype.bas)

                if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                    sol=task.getprimalobj(soltype.bas)
                elif (solsta == mosek.solsta.dual_infeas_cer ):
                    print("Dual infeasibility certificate found.\n")
                    sol="No factible"
                elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                    print("Primal infeasibility certificate found.\n")
                    sol="No factible"
                elif solsta == mosek.solsta.unknown:
                    print("Unknown solution status")
                    sol="No factible"
                else:
                    print("Other solution status")
                    sol="Other solution status"
    
    return sol


**PG

\begin{equation}
\begin{aligned}
  \underset{\delta_1,\delta_2,y \in \mathbb{R}^{|\Omega|}}{\underset{\gamma_1,\gamma_2, \lambda,x \in \mathbb{R}}{\text{min}}}  & \quad cx+\gamma_1 + \delta_1^T\theta-\delta_2^T\theta+2\epsilon\lambda-Ky\\
    \text{s.t.} \;\; & \gamma_1\mathbbm{1}_A+\gamma_2\mathbbm{1}_{A^c}+\delta_1-\delta_2\geq -py\\
    \; & \lambda\mathbbm{1}-\delta_1\geq \textbf{0}\\
    \; & \lambda\mathbbm{1}-\delta_2\geq \textbf{0}\\
    \; & x\mathbbm{1}-y\geq \textbf{0}\\
    \; & y(i)=0, \quad \forall \omega_i \in A^c\\
    \; & \lambda,x,\delta_1(i),\delta_2(i),y(i) \geq 0, \quad i=1,\dots,|\Omega|\\
\end{aligned}
\label{dual de f(x,A)}
\end{equation}\\

In [10]:
#Función que devuelve la solución x_A para un conjunto A 

def xA(A,O):
    
    s=size(D)
    
    matrizA=ones((5*s,4+3*s))
    K=0.0
    
    indicA=zeros(s)
    for i in range(size(A)):
        cual=D.index(A[i])
        indicA[cual]=1

    epsilon=gamma(A)

    with mosek.Env() as env:
        with env.Task(0, 0) as task:
        
            # Bound keys for constraints
            bkc = [mosek.boundkey.lo]*4*s+[mosek.boundkey.fx]*s

            # Bound values for constraints

            blc = [0]*5*s
            buc = [inf]*4*s+[0]*s

            # Bound keys for variables
            bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1)+[mosek.boundkey.ra]*s+[mosek.boundkey.lo]

            # Bound values for variables
            blx = [-inf]*2+[0]*(3*s+2)
            bux = [+inf]*(3+2*s)+D+[+inf]

            # Objective coefficients
            mO=[]
            for i in range(size(O)):
                mO.append(-O[i])            
            
            coef = [1]+[0]+[epsilon]+O+mO+[K]*s+[c]

            # Below is the sparse representation of the A
            # matrix stored by column.

            numvar = len(bkx)
            numcon = len(bkc)

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)

            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            #Append constraint matrix 
            task.putaijlist(range(0,s),[0]*s,indicA)
            task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
            task.putaijlist(range(0,s),range(3,3+s),ones(s))
            task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s)) 
            task.putaijlist(range(0,s),range(3+2*s,3+3*s),p*ones(s))
            task.putaijlist(range(s,2*s),[2]*s,ones(s))
            task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
            task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))
            task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
            task.putaijlist(range(3*s,4*s),range(3+2*s,3+3*s),-ones(s))
            task.putaijlist(range(3*s,4*s),[3+3*s]*s,ones(s))
            task.putaijlist(range(4*s,5*s),range(3+2*s,3+3*s),(ones(s)-indicA))
        
        
            for i in range(numcon):
                for j in range(numvar):
                    matrizA[i,j]=task.getaij(i,j) 

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, coef[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])  


            # Set the bounds on constraints.
            # blc[i] <= constraint_i <= buc[i]
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)

            # Get status information about the solution
            solsta = task.getsolsta(mosek.soltype.bas)

            if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                xx = [0.] * numvar
                task.getxx(mosek.soltype.bas, xx) # Request the basic solution.
                sol=xx[numvar-1]
                
            elif (solsta == mosek.solsta.dual_infeas_cer ):
                print("Dual infeasibility certificate found.\n")
                sol="No factible"
            elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                print("Primal infeasibility certificate found.\n")
                sol="No factible"
            elif solsta == mosek.solsta.unknown:
                print("Unknown solution status")
                sol="No factible"
            else:
                print("Other solution status")
                sol="Other solution status"
    
    return sol


\begin{equation}
\begin{aligned}
  \underset{\delta_1,\delta_2,y \in \mathbb{R}^{|A|}}{\underset{\gamma_1,\gamma_2, \lambda,x \in \mathbb{R}}{\text{min}}}  & \quad cx+\gamma_1 + \delta_1^T\theta-\delta_2^T\theta+2\epsilon\lambda-Ky\\
    \text{s.t.} \;\; & \gamma_1\mathbbm{1}_{A}+\gamma_2\mathbbm{1}_{A^c}+\delta_1-\delta_2\geq -py\\
    \; & \lambda\mathbbm{1}-\delta_1\geq \textbf{0}\\
    \; & \lambda\mathbbm{1}-\delta_2\geq \textbf{0}\\
    \; & x\mathbbm{1}-y\geq \textbf{0}\\
    \; & y(i)=0, \quad \forall \omega_i \in A^c\\
    \; & \lambda,x,\delta_1(i),\delta_2(i),y(i) \geq 0, \quad i=1,\dots,|\Omega|\\
\end{aligned}
\label{dual de f(x,A)}
\end{equation}\\

Acá $\gamma_2$ sobra ... tomo $\mathbbm{1}_{A}= \mathbbm{1}$ y $\mathbbm{1}_{A^c}=\textbf{0}$

In [11]:
#Función que devuelve la solución x^* para un conjunto A con distribución Q

def xARed(A,Q):
    
    s=size(A)
    
    matrizA=ones((5*s,4+3*s))
    K=0.0
    
    indicA=ones(s)
    epsilon=2*eps

    with mosek.Env() as env:
        with env.Task(0, 0) as task:
        
            # Bound keys for constraints
            bkc = [mosek.boundkey.lo]*4*s+[mosek.boundkey.fx]*s

            # Bound values for constraints

            blc = [0]*5*s
            buc = [inf]*4*s+[0]*s

            # Bound keys for variables
            bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1)+[mosek.boundkey.ra]*s+[mosek.boundkey.lo]

            # Bound values for variables
            blx = [-inf]*2+[0]*(3*s+2)
            bux = [+inf]*(3+2*s)+D+[+inf]

            # Objective coefficients
            mO=[]
            for i in range(size(Q)):
                mO.append(-Q[i])            
            
            coef = [1]+[0]+[epsilon]+Q+mO+[K]*s+[c]

            # Below is the sparse representation of the A
            # matrix stored by column.

            numvar = len(bkx)
            numcon = len(bkc)

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)

            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            #Append constraint matrix 
            task.putaijlist(range(0,s),[0]*s,indicA)
            task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
            task.putaijlist(range(0,s),range(3,3+s),ones(s))
            task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s)) 
            task.putaijlist(range(0,s),range(3+2*s,3+3*s),p*ones(s))
            task.putaijlist(range(s,2*s),[2]*s,ones(s))
            task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
            task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))
            task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
            task.putaijlist(range(3*s,4*s),range(3+2*s,3+3*s),-ones(s))
            task.putaijlist(range(3*s,4*s),[3+3*s]*s,ones(s))
            task.putaijlist(range(4*s,5*s),range(3+2*s,3+3*s),(ones(s)-indicA))
        
        
            for i in range(numcon):
                for j in range(numvar):
                    matrizA[i,j]=task.getaij(i,j) 

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, coef[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])  


            # Set the bounds on constraints.
            # blc[i] <= constraint_i <= buc[i]
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)

            # Get status information about the solution
            solsta = task.getsolsta(mosek.soltype.bas)

            if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                xx = [0.] * numvar
                task.getxx(mosek.soltype.bas, xx) # Request the basic solution.
                sol=xx[numvar-1]
                
            elif (solsta == mosek.solsta.dual_infeas_cer ):
                print("Dual infeasibility certificate found.\n")
                sol="No factible"
            elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                print("Primal infeasibility certificate found.\n")
                sol="No factible"
            elif solsta == mosek.solsta.unknown:
                print("Unknown solution status")
                sol="No factible"
            else:
                print("Other solution status")
                sol="Other solution status"
    
    return sol

In [12]:
#Función que devuelve el valor óptimo del problema robusto para un conjunto A con distribución Q

def VOR(A,Q):
    
    s=size(A)
    
    matrizA=ones((5*s,4+3*s))
    K=0.0
    
    indicA=ones(s)
    epsilon=2*eps

    
    with mosek.Env() as env:
        with env.Task(0, 0) as task:
        
            # Bound keys for constraints
            bkc = [mosek.boundkey.lo]*4*s+[mosek.boundkey.fx]*s

            # Bound values for constraints

            blc = [0]*5*s
            buc = [inf]*4*s+[0]*s

            # Bound keys for variables
            bkx = [mosek.boundkey.fr]*2+[mosek.boundkey.lo]*(2*s+1)+[mosek.boundkey.ra]*s+[mosek.boundkey.lo]

            # Bound values for variables
            blx = [-inf]*2+[0]*(3*s+2)
            bux = [+inf]*(3+2*s)+D+[+inf]

            # Objective coefficients
            mO=[]
            for i in range(size(Q)):
                mO.append(-Q[i])            
            
            coef = [1]+[0]+[epsilon]+Q+mO+[K]*s+[c]

            # Below is the sparse representation of the A
            # matrix stored by column.

            numvar = len(bkx)
            numcon = len(bkc)

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)

            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            #Append constraint matrix 
            task.putaijlist(range(0,s),[0]*s,indicA)
            task.putaijlist(range(0,s),[1]*s,(ones(s)-indicA))
            task.putaijlist(range(0,s),range(3,3+s),ones(s))
            task.putaijlist(range(0,s),range(3+s,3+2*s),-ones(s)) 
            task.putaijlist(range(0,s),range(3+2*s,3+3*s),p*ones(s))
            task.putaijlist(range(s,2*s),[2]*s,ones(s))
            task.putaijlist(range(s,2*s),range(3,3+s),-ones(s))
            task.putaijlist(range(2*s,3*s),range(3+s,3+2*s),-ones(s))
            task.putaijlist(range(2*s,3*s),[2]*s,ones(s))
            task.putaijlist(range(3*s,4*s),range(3+2*s,3+3*s),-ones(s))
            task.putaijlist(range(3*s,4*s),[3+3*s]*s,ones(s))
            task.putaijlist(range(4*s,5*s),range(3+2*s,3+3*s),(ones(s)-indicA))
        
        
            for i in range(numcon):
                for j in range(numvar):
                    matrizA[i,j]=task.getaij(i,j) 

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, coef[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])  


            # Set the bounds on constraints.
            # blc[i] <= constraint_i <= buc[i]
            for i in range(numcon):
                task.putconbound(i, bkc[i], blc[i], buc[i])

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)

            # Get status information about the solution
            solsta = task.getsolsta(mosek.soltype.bas)

            if (solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal): 
                xx = [0.] * numvar
                task.getxx(mosek.soltype.bas, xx) # Request the basic solution.
                sol=task.getprimalobj(soltype.bas)
                
            elif (solsta == mosek.solsta.dual_infeas_cer ):
                print("Dual infeasibility certificate found.\n")
                sol="No factible"
            elif(solsta == mosek.solsta.prim_infeas_cer or solsta == mosek.solsta.near_prim_infeas_cer):
                print("Primal infeasibility certificate found.\n")
                sol="No factible"
            elif solsta == mosek.solsta.unknown:
                print("Unknown solution status")
                sol="No factible"
            else:
                print("Other solution status")
                sol="Other solution status"
    
    return sol



In [13]:
#Optimo no robusto: 

epstemp=eps

eps=0
xNR=xA(D,P)
optimoNR=VOR(D,P)

print(xNR)
print(optimoNR)

eps=epstemp

52.00000018950678
-321.5199983345242


In [16]:
xopt=xA(D,P)
vopt=VOR(D,P)

print(xopt)
print(vopt)
print(size(D))

50.99999999763962
-297.08000001394294
35


**Clustering con la distancia 1

$d_1(A,B):=2f(X_{A\cup B}^*,A\cup B)-f(X_A^*,A)-f(X_B^*,B)$

In [17]:
#Distancia 1
M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])


print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x* No Robusto:",xNR)
print("x Robusto:", xopt)
print("           ")



contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=2*funcionf(xab,AuB)-funcionf(xa,A)-funcionf(xb,B)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                
    print(m)              
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    

    
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")
    
    
    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=2*funcionf(xab,AuB)-funcionf(xa,A)-funcionf(xb,B)
            M[(t-1),i]=M[i,(t-1)]

        

clusters:  [[31], [32], [34], [35], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [65], [66], [67], [68]]
Distribución clusters:  [0.006666666666666667, 0.006666666666666667, 0.006666666666666667, 0.013333333333333334, 0.006666666666666667, 0.013333333333333334, 0.006666666666666667, 0.03333333333333333, 0.013333333333333334, 0.02, 0.02666666666666667, 0.02, 0.04, 0.04, 0.06666666666666667, 0.060000000000000005, 0.05333333333333334, 0.03333333333333333, 0.08, 0.05333333333333334, 0.04, 0.04666666666666667, 0.060000000000000005, 0.04666666666666667, 0.04666666666666667, 0.03333333333333333, 0.02666666666666667, 0.006666666666666667, 0.02, 0.013333333333333334, 0.02, 0.006666666666666667, 0.02, 0.006666666666666667, 0.006666666666666667]
x* No Robusto: 52.00000018950678
x Robusto: 50.99999999763962
           
7.0
clusters:  [[34], [35], [37], [38], [39], [40], [41], [42], 

7.0
clusters:  [[49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [65], [66], [67], [68], [31, 32], [34, 35], [37, 38], [39, 40], [41, 42], [43, 44], [45, 46], [47, 48]]
Representantes:  [31, 34, 37, 39, 41, 43, 45, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68]
Distribución clusters:  [0.05333333333333334, 0.03333333333333333, 0.08, 0.05333333333333334, 0.04, 0.04666666666666667, 0.060000000000000005, 0.04666666666666667, 0.04666666666666667, 0.03333333333333333, 0.02666666666666667, 0.006666666666666667, 0.02, 0.013333333333333334, 0.02, 0.006666666666666667, 0.02, 0.006666666666666667, 0.006666666666666667, 0.013333333333333334, 0.02, 0.02, 0.04, 0.03333333333333333, 0.04666666666666667, 0.08, 0.12666666666666668]
x*:  50.99999999763962
x* Reducido: 41.99999998078036
Medida de Desempeño Robusto: 0.16143799585615018
Medida de Desempeño No Robusto: 0.17392385688335318
           
7.0
clusters:  [[51], [52], [53],

7.0
clusters:  [[31, 32], [34, 35], [37, 38], [39, 40], [41, 42], [43, 44], [45, 46], [47, 48], [49, 50], [51, 52], [53, 54, 55], [56, 57], [58, 59], [60, 61], [62, 63], [65, 66], [67, 68]]
Representantes:  [31, 34, 37, 39, 41, 43, 45, 47, 49, 51, 53, 56, 58, 60, 62, 65, 67]
Distribución clusters:  [0.013333333333333334, 0.02, 0.02, 0.04, 0.03333333333333333, 0.04666666666666667, 0.08, 0.12666666666666668, 0.08666666666666667, 0.13333333333333333, 0.14666666666666667, 0.09333333333333334, 0.06, 0.02666666666666667, 0.03333333333333333, 0.02666666666666667, 0.013333333333333334]
x*:  50.99999999763962
x* Reducido: 41.99999999999999
Medida de Desempeño Robusto: 0.12427628925612656
Medida de Desempeño No Robusto: 0.14978850096245216
           
14.0
clusters:  [[31, 32], [34, 35], [37, 38], [39, 40], [45, 46], [47, 48], [49, 50], [51, 52], [53, 54, 55], [56, 57], [58, 59], [60, 61], [62, 63], [65, 66], [67, 68], [41, 42, 43, 44]]
Representantes:  [31, 34, 37, 39, 41, 45, 47, 49, 51, 53, 5

44.919999972024556
clusters:  [[60, 61, 62, 63, 65, 66, 67, 68], [53, 54, 55, 56, 57, 58, 59, 31, 32, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]]
Representantes:  [31, 60]
Distribución clusters:  [0.1, 0.8999999999999999]
x*:  50.99999999763962
x* Reducido: 32.0
Medida de Desempeño Robusto: 0.2540729770109076
Medida de Desempeño No Robusto: 0.3070415490355015
           
102.6800000145891
clusters:  [[60, 61, 62, 63, 65, 66, 67, 68, 53, 54, 55, 56, 57, 58, 59, 31, 32, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]]
Representantes:  [31]
Distribución clusters:  [1.0]
x*:  50.99999999763962
x* Reducido: 31.0
Medida de Desempeño Robusto: 0.2695570217119447
Medida de Desempeño No Robusto: 0.3250808623909508
           


**Clustering con la distancia 2

$d_2(A,B):=f(X_{A\cup B}^*,B)-f(X_B^*,B)+f(X_{A\cup B}^*,A)-f(X_A^*,A)$

In [12]:
#Distancia 2
M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

xopt=xA(D,P)
print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x* No Robusto:",xNR)
print("x Robusto:", xopt)
print("           ")


contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xab,B)-funcionf(xb,B)+funcionf(xab,A)-funcionf(xa,A)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                
    print(m)              
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    
    m=100000000
    x=0

    for i in range(size(rep)):
        if objetivo(rep[i],rep,Q)<m:
            m=objetivo(rep[i],rep,Q)
            x=rep[i]
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")
    
    
    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xab,B)-funcionf(xb,B)+funcionf(xab,A)-funcionf(xa,A)
            M[(t-1),i]=M[i,(t-1)]



clusters:  [[11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x* No Robusto: 20.999999999997197
x Robusto: 19.999999831268106
           
7.0
clusters:  [[13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30], [11, 12]]
Representantes:  [11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30]
Distribución clusters:  [0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04, 0.06]
x*:  19.999999831268106
x* Reducido: 18.99999982613958
Medida de Desempeño Robusto: 0.06814868429566999
x No Robusto del Reducido:  21
Medida de Desempeño No Robusto: 0.07135777997804618
           
7.0
clusters:  [[15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30], [11, 12], [13, 14]]
Representantes:  [1

**Clustering con la distancia 3

$d_3(A,B):=f(X_A^*,A\cup B)-f(X_A^*,A)+f(X_B^*,A\cup B)-f(X_B^*,B)$

In [13]:
#Distancia 3
M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

xopt=xA(D,P)
print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x* No Robusto:",xNR)
print("x Robusto:", xopt)
print("           ")


contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xb,AuB)-funcionf(xb,B)+funcionf(xa,AuB)-funcionf(xa,A)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                
    print(m)              
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")
    
    
    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xb,AuB)-funcionf(xb,B)+funcionf(xa,AuB)-funcionf(xa,A)
            M[(t-1),i]=M[i,(t-1)]



clusters:  [[11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x* No Robusto: 20.999999999997197
x Robusto: 19.999999831268106
           
10.559999999999974
clusters:  [[11], [12], [13], [14], [15], [17], [18], [21], [22], [23], [24], [25], [26], [27], [29], [30], [19, 20]]
Representantes:  [11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 29, 30]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04, 0.2]
x*:  19.999999831268106
x* Reducido: 20.99999999723829
Medida de Desempeño Robusto: 0.001822152870982258
x No Robusto del Reducido:  24
Medida de Desempeño No Robusto: 0.43508424182229033
           
9.959999998605213
clusters:  [[11], [12], [13], [14], [15], [17], [21], [22], [23], [24], [25], [26], [27], [29], [30], [18,

**Clustering con la distancia 4

$d_4(A,B):=f(X_A^*,A\cup B)+f(X_B^*,A\cup B)-2f(X_{A\cup B}^*,A\cup B)$

In [15]:
#Distancia 4
M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

xopt=xA(D,P)
print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x* No Robusto:",xNR)
print("x Robusto:", xopt)
print("           ")


contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xb,AuB)+funcionf(xa,AuB)-2*funcionf(xab,AuB)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                
    print(m)              
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
    
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")
    
    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xb,AuB)+funcionf(xa,AuB)-2*funcionf(xab,AuB)
            M[(t-1),i]=M[i,(t-1)]

clusters:  [[11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x* No Robusto: 20.999999999997197
x Robusto: 19.999999831268106
           
3.5599999999999454
clusters:  [[11], [12], [13], [14], [15], [17], [18], [21], [22], [23], [24], [25], [26], [27], [29], [30], [19, 20]]
Representantes:  [11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 29, 30]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04, 0.2]
x*:  19.999999831268106
x* Reducido: 20.99999999723829
Medida de Desempeño Robusto: 0.001822152870982258
x No Robusto del Reducido:  24
Medida de Desempeño No Robusto: 0.43508424182229033
           
2.959999998604758
clusters:  [[11], [12], [13], [14], [15], [17], [21], [22], [23], [24], [25], [26], [27], [29], [30], [18,

**Clustering con la distancia 5

$d_5(A,B):=f(X_A^*,B)-f(X_B^*,B)+f(X_B^*,A)-f(X_A^*,A)$

In [13]:
#Distancia 5
M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])

xopt=xA(D,P)
print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x* No Robusto:",xNR)
print("x Robusto:", xopt)
print("           ")


contador=size(Z)

for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                AuB=Z[i]+Z[j]
                A=Z[i]
                B=Z[j]
                xab=xA(AuB,P)
                xa=xA(A,P)
                xb=xA(B,P)
                M[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
                M[j,i]=M[i,j]
                               

while contador>1:
        
    
    m=1000000000
    im=0
    jm=0
    
    for i in range(contador):
        for j in range(i,contador):
            if i!=j and M[i,j]<m:
                m=M[i,j]
                im=i
                jm=j
                
    print(m)              
    
    Ztemp=[]
    for k in range(contador):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])
            
            
    C=Z[im]+Z[jm]
    Ztemp.append(C)
    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(min(Z[k]))
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")
    
    
    t=contador
    MV=M
    M=zeros((t,t))
    
    k=0
    for i in range(t+1):
        l=0
        if i!=im and i!=jm:
            for j in range((t+1)):
                if j!=im and j!=jm:
                    M[k,l]=MV[i,j]
                    l=l+1
            k=k+1

    for i in range(t-1):
            AuB=Z[i]+Z[t-1]
            A=Z[i]
            B=Z[t-1]
            xab=xA(AuB,P)
            xa=xA(A,P)
            xb=xA(B,P)
            M[i,(t-1)]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
            M[(t-1),i]=M[i,(t-1)]

clusters:  [[9], [11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [32], [33]]
Distribución clusters:  [0.04, 0.04, 0.04, 0.02, 0.02, 0.04, 0.06, 0.08, 0.08, 0.04, 0.1, 0.08, 0.12000000000000001, 0.04, 0.06, 0.02, 0.04, 0.02, 0.02, 0.02, 0.02]
x* No Robusto: 21.999999566765663
x Robusto: 20.99999999059844
           
12.0
clusters:  [[9], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [32], [33], [11, 12]]
Representantes:  [9, 11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33]
Distribución clusters:  [0.04, 0.02, 0.02, 0.04, 0.06, 0.08, 0.08, 0.04, 0.1, 0.08, 0.12000000000000001, 0.04, 0.06, 0.02, 0.04, 0.02, 0.02, 0.02, 0.02, 0.08]
x*:  20.99999999059844
x* Reducido: 20.0
Medida de Desempeño Robusto: 0.030655801372628782
Medida de Desempeño No Robusto: 0.03702445674832182
           
12.0
clusters:  [[9], [15], [17], [18], [19], [20], [21], [22], [23], [2

63.239999999996655
clusters:  [[32, 33], [9, 11, 12, 13, 14], [15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 27, 28]]
Representantes:  [9, 15, 32]
Distribución clusters:  [0.04, 0.15999999999999998, 0.8000000000000003]
x*:  20.99999999059844
x* Reducido: 12.0
Medida de Desempeño Robusto: 0.25261932483734306
Medida de Desempeño No Robusto: 0.3152173918143606
           
109.20000000088737
clusters:  [[32, 33], [9, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 27, 28]]
Representantes:  [9, 32]
Distribución clusters:  [0.04, 0.9600000000000002]
x*:  20.99999999059844
x* Reducido: 11.0
Medida de Desempeño Robusto: 0.28560341486269397
Medida de Desempeño No Robusto: 0.3542798917852679
           
130.75200000050754
clusters:  [[32, 33, 9, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 27, 28]]
Representantes:  [9]
Distribución clusters:  [1.0]
x*:  20.99999999059844
x* Reducido: 9.0
Medida de Desempeño Robusto: 0.38882421423614505
Medida de Desempeño No 

**Clustering con la distancia 6

$d_6(A,B):=\text{max}\{d_5(\omega_1,\omega_2):\omega_1\in A, \omega_2 \in B\}$

In [17]:
#Distancia 6
Z=[]
for i in range(s):
    Z.append([D[i]])
    

D5=zeros((s,s))
for i in range(s):
    for j in range(i,s):
        if i!=j:
            A=[D[i]]
            B=[D[j]]
            xa=xA(A,P)
            xb=xA(B,P)
            D5[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
            D5[j,i]=D5[i,j]
        

xopt=xA(D,P)
print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x*:",xNR)
print("x Robusto:", xopt)
print("           ")

contador=size(Z)

while contador>1:
    
    m=1000000000
    im=0
    jm=0

    M=zeros((size(Z),size(Z)))
    for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                A=Z[i]
                B=Z[j]
                distancia=0
                for k in range(size(A)):
                    for l in range(size(B)):
                        posa=D.index(A[k])
                        posb=D.index(B[l])
                        if D5[posa,posb]>distancia:
                            distancia=D5[posa,posb]
                            M[i,j]=distancia
                M[j,i]=M[i,j]
                if M[i,j]<m: 
                    m=M[i,j];
                    im=i
                    jm=j
              
  
    print(m)
    C=Z[im]+Z[jm]
    Ztemp=[C]
    for k in range(size(Z)):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])

    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(Z[k][0])
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")

clusters:  [[11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x*: 20.999999999997197
x Robusto: 19.999999831268106
           
12.0
clusters:  [[11, 12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Representantes:  [11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30]
Distribución clusters:  [0.06, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x*:  19.999999831268106
x* Reducido: 19.0
Medida de Desempeño Robusto: 0.05721573908335224
x No Robusto del Reducido:  21
Medida de Desempeño No Robusto: 0.03567888998787512
           
12.0
clusters:  [[13, 14], [11, 12], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Representantes:  [11, 13, 15, 17, 18, 19,

**Clustering con la distancia 7

$d_7(A,B):=\dfrac{1}{|A||B|}\sum_{\omega_1\in A} \sum_{\omega_2 \in B}d_5(\omega_1,\omega_2)$

In [18]:
#Distancia 7
M=zeros((s,s))
Z=[]
for i in range(s):
    Z.append([D[i]])
    
    

D5=zeros((s,s))
for i in range(s):
    for j in range(i,s):
        if i!=j:
            A=[D[i]]
            B=[D[j]]
            xa=xA(A,P)
            xb=xA(B,P)
            D5[i,j]=funcionf(xa,B)-funcionf(xb,B)+funcionf(xb,A)-funcionf(xa,A)
            D5[j,i]=M[i,j]
        

xopt=xA(D,P)
print("clusters: ",Z)
print("Distribución clusters: ", P)
print("x*:",xNR)
print("x Robusto:", xopt)
print("           ")

contador=size(Z)

while contador>1:
    
    m=1000000000
    im=0
    jm=0

    M=zeros((size(Z),size(Z)))
    for i in range(size(Z)):
        for j in range(i,size(Z)):
            if i!=j:
                A=Z[i]
                B=Z[j]
                distancia=0
                for k in range(size(A)):
                    for l in range(size(B)):
                        posa=D.index(A[k])
                        posb=D.index(B[l])
                        distancia=distancia+D5[posa,posb]
                
                dist=distancia/(size(A)*size(B))
                M[i,j]=dist
                M[j,i]=M[i,j]
                if M[i,j]<m: 
                    m=M[i,j];
                    im=i
                    jm=j
              
  
    print(m)
    C=Z[im]+Z[jm]
    Ztemp=[C]
    for k in range(size(Z)):
        if k!=im and k!=jm:
            Ztemp.append(Z[k])

    Z=Ztemp
    contador=contador-1
    print("clusters: ",Z)
    
    rep=[]
    Q=[]
    
    for k in range(contador):
        pk=0
        for l in range(size(Z[k])):
            pk=pk+P[D.index(Z[k][l])]
        
        rep.append(Z[k][0])
        Q.append(pk)
    
    rep.sort()
    
    print("Representantes: ",rep)
    print("Distribución clusters: ", Q)
    
    m=100000000
    x=0

    for i in range(size(rep)):
        if objetivo(rep[i],rep,Q)<m:
            m=objetivo(rep[i],rep,Q)
            x=rep[i]
    
    vopt=VOR(D,P)
    medida1=abs(VOR(rep,Q)-vopt)/abs(vopt)
    
    etemp=eps
    eps=0
    oh=VOR(rep,Q)
    eps=etemp
    
    obje=optimoNR
    medida2=abs(oh-obje)/abs(obje)
    

    print("x*: ", xopt)
    print("x* Reducido:",xARed(rep,Q))
    print("Medida de Desempeño Robusto:", medida1)
    print("Medida de Desempeño No Robusto:", medida2)
    print("           ")

clusters:  [[11], [12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Distribución clusters:  [0.04, 0.02, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x*: 20.999999999997197
x Robusto: 19.999999831268106
           
12.0
clusters:  [[11, 12], [13], [14], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Representantes:  [11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30]
Distribución clusters:  [0.06, 0.02, 0.06, 0.02, 0.08, 0.1, 0.06, 0.14, 0.08, 0.06, 0.06, 0.04, 0.08, 0.04, 0.02, 0.04, 0.04]
x*:  19.999999831268106
x* Reducido: 19.0
Medida de Desempeño Robusto: 0.05721573908335224
x No Robusto del Reducido:  21
Medida de Desempeño No Robusto: 0.03567888998787512
           
12.0
clusters:  [[13, 14], [11, 12], [15], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [29], [30]]
Representantes:  [11, 13, 15, 17, 18, 19,